
## Appliance Energy Prediction

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing, model_selection, metrics
import warnings
warnings.filterwarnings("ignore")

# Reading the data

In [ ]:
import os 
os.getcwd()

In [ ]:
df = pd.read_csv("/home/jupyter/energydata_complete.csv", parse_dates=['date'])

In [ ]:
df.head()

In [ ]:
df.info()

# Data Exploration 

In [ ]:
df.describe()

In [ ]:
# presence of 0 whithin the df
zero = 0
for cols in df.columns:
    if zero in df[cols]:
        print('Found in '+cols) 

### Consideration

All variables present at least a zero in their rilevation. This can be attributed to an error in the process of acquisiton information. 
Temperature, Humidity, Windspeed, Dew point obviously cannot be as low as 0 not even just for a single entry. 

In [ ]:
# Check for Null values in df
df.isnull().sum().sort_values(ascending = True)

### This dataset is fully filled. 
### But are we sure that all the entries are useful? 

In [ ]:
# Find outliers
sorted_appliances = df.sort_values('Appliances',ascending=False)
print("The number of the 0,1% top values of Appliances' load is",
      len(sorted_appliances.head(len(sorted_appliances)//1000)),"and they have power load higher than",
      sorted_appliances.Appliances[19], "Wh.")

# boxplot appliances
sns.set(style="whitegrid")
ax = sns.boxplot(sorted_appliances.Appliances)

In [ ]:
# Removing outliers
df = df.dropna()
df = df.drop(df[(df.Appliances>790)|(df.Appliances<0)].index)

In [ ]:
# Columns based on type 

col_temp = ["T1","T2","T3","T4","T5","T6","T7","T8","T9"]

col_hum = ["RH_1","RH_2","RH_3","RH_4","RH_5","RH_6","RH_7","RH_8","RH_9", "RH_out"]

col_weather = ["T_out", "Tdewpoint","RH_out","Press_mm_hg",
                "Windspeed","Visibility"] 
col_light = ["lights"]

col_randoms = ["rv1", "rv2"]

col_target = ["Appliances"]

In [ ]:
# variables divisions 
#feature_vars = [col_temp + col_hum + col_weather + col_light + col_randoms ]
#target_var = [col_target]

In [ ]:
#feature_vars.describe()

In [ ]:
#target_var.describe()

### Observations 

1. Lights - More than 75% of the column is filled with zeros. I can't use any technique to replace those zeros, thus, for now it is bettere to remove it. In addition I can't use any fillied techniques to preserv it and it will negatively inflict the model I am going to apply.  

2. Humidiy  - As expected the most humid place within the house is RH_5 (Bathroom) with a range between 29.82% to 96.32%. 
 

In [ ]:
df = df.drop(['lights'], axis=1)

# Data Visualization

In [ ]:
# Set the style
plt.style.use('fivethirtyeight')

In [ ]:
# Set up the plotting layout
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(nrows=2, ncols=2, figsize = (35,15))
fig.autofmt_xdate(rotation = 45)

# TEMPERATURE

# Living Room 
ax1.plot(df.date, df['T2'], linewidth=1, alpha=.7, label = "Living Room")
ax1.set_xlabel('Nov 1st 2016 - May 27th 2016'); ax1.set_ylabel('Temp in Celsius'); ax1.set_title('Temperature Trend')
# Parents Room 
ax1.plot(df.date, df['T9'], linewidth=1, alpha=.7, label = "Parents Room")
ax1.set_xlabel('Nov 1st 2016 - May 27th 2016');
# Ironing Room 
ax1.plot(df.date, df['T7'], linewidth=1, alpha=.7, label = "Ironing Room")
ax1.set_xlabel('Nov 1st 2016 - May 27th 2016'); 
ax1.legend()

# HUMIDITY Outside vs HUMIDITY Wheather Station

# Outiside
ax2.plot(df.date, df.RH_6, linewidth=1, alpha=.7, label = "North Outside House")
ax2.set_xlabel('Nov 1st 2016 - May 27th 2016'); ax2.set_ylabel('Humidity in %'); ax2.set_title('Humidity Trend')
# Weather Station
ax2.plot(df.date, df.RH_out, linewidth=1, alpha=.7, label = "Weather Station")
ax2.set_xlabel('Nov 1st 2016 - May 27th 2016')
ax2.legend()

# OUTSIDE HUMIDITY (maybe fog?) vs VISIBILITY 

# Windspeed
ax3.plot(df.date, df.RH_out, linewidth=1, alpha=.7, label="Outside Humidity")
ax3.set_xlabel('Nov 1st 2016 - May 27th 2016'), ax3.set_title('Humidity vs Visibility')
# Visibility
ax3.plot(df.date, df.Visibility, linewidth=1, alpha=.7, label="Visibility")
ax3.set_xlabel('Nov 1st 2016 - May 27th 2016')
ax3.legend()


# TEMPERATURE Outside vs TEMPERATURE Wheather Station
# Outiside
ax4.plot(df.date, df.T6, linewidth=1, alpha=.7, label = "North Outside House")
ax4.set_xlabel('Nov 1st 2016 - May 27th 2016'); ax2.set_ylabel('Temp in Celsius'); ax4.set_title('Temperature Trend')
# Weather Station
ax4.plot(df.date, df.T_out, linewidth=1, alpha=.7, label = "Weather Station")
ax4.set_xlabel('Nov 1st 2016 - May 27th 2016')
ax4.legend()


plt.tight_layout(pad=2)

In [ ]:
# Histogram of all the features to understand the distribution
df.hist(bins = 20 , figsize= (12,16))

In [ ]:
# focused displots for RH_6 , RH_out , Visibility , Windspeed due to irregular distribution
f, ax = plt.subplots(2,2,figsize=(12,8))
vis1 = sns.distplot(df["RH_6"],bins=10, ax=ax[0][0])
vis2 = sns.distplot(df["RH_out"],bins=10, ax=ax[0][1])
vis3 = sns.distplot(df["Visibility"],bins=10, ax=ax[1][0])
vis4 = sns.distplot(df["Windspeed"],bins=10, ax=ax[1][1])

## Awesome, another skewed distribution!

In [ ]:
# Distribution of values in Applainces column
f = plt.figure(figsize=(12,5))
plt.xlabel('Appliance consumption in Wh')
plt.ylabel('Frequency')
sns.distplot(df["Appliances"] , bins=10 ) 

In [ ]:
#log appliances
df['log_appliances'] = np.log(df.Appliances)

In [ ]:
# Distribution of values in log_appliances column
f = plt.figure(figsize=(12,5))
plt.xlabel('Appliance consumption in Wh')
plt.ylabel('Frequency')
sns.distplot(df["log_appliances"] , bins=10 ) ;

In [ ]:
# Drop date column, I can add it later on
df = df.drop("date", axis=1)

In [ ]:
df.columns

# Let's remove humidity's columns for this first trial, I can add it later on 

In [ ]:
#df = df.drop(col_hum, axis=1)

## Features and Targets and Convert Data to Arrays

In [ ]:
# Labels are the values we want to predict
labels = np.array(df['log_appliances'])

# Remove the labels from the features
# axis 1 refers to the columns
features= df.drop(['log_appliances', 'Appliances'], axis = 1)

# Saving feature names for later use
feature_list = list(features.columns)

# Convert to numpy array
features = np.array(features)

## Establish Baseline

Our baseline is the error we would get if we simply predict the average consumption in Watt/hour for Appliances.

In [ ]:
### TODO: TROVARE UNA STRATEGIA PER STABILIRE UNA BASELINE

In [ ]:
# The baseline predictions is the average value of Watt/hour
#baseline_preds = test_features[:, feature_list.index('Appliances')]
# Baseline errors, and display average baseline error
#baseline_errors = abs(baseline_preds - test_labels)
#print('Average baseline error: ', round(np.mean(baseline_errors), 2), 'degrees.')

In [ ]:
from sklearn.model_selection import train_test_split
# Split the data into training and testing sets
# with test_siz = 0.33 I get a Training Set to small, let's set it to .025
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size = 0.25, random_state = 1234)

In [ ]:
print('Training Features Shape:', train_features.shape)
print('Training Labels Shape:', train_labels.shape)
print('Testing Features Shape:', test_features.shape)
print('Testing Labels Shape:', test_labels.shape)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 1000, random_state = 1234)
# Train the model on training data
rf.fit(train_features, train_labels)

In [ ]:
# Use the forest's predict method on the test data
predictions = rf.predict(test_features)
# Calculate the absolute errors
errors = abs(predictions - test_labels)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')

In [ ]:
# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / test_labels)
# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

In [ ]:
# Select one tree from the forest
from sklearn.tree import export_graphviz
import pydot
tree = rf.estimators_[5]
# Export in dot file
export_graphviz(tree, out_file = 'tree.dot', feature_names = feature_list, rounded = True, precision = 1)
# Create a graph
(graph, ) = pydot.graph_from_dot_file('tree.dot')
# Convert it in png
graph.write_png('tree.png')

In [ ]:
print('The depth of this tree is:', tree.tree_.max_depth)

### The image is quite large with 45 layer's level!
### Let's focus on just once

In [ ]:
# Select just 3 levels among 35
rf_small = RandomForestRegressor(n_estimators=10, max_depth = 3)
rf_small.fit(train_features, train_labels)
# Select just one small
tree_small = rf_small.estimators_[1]
# Export as png
export_graphviz(tree_small, out_file = 'small_tree.dot', feature_names = feature_list, rounded = True, precision = 1)
(graph, ) = pydot.graph_from_dot_file('small_tree.dot')
graph.write_png('small_tree.png')

### Observation: there is a prevalence of Temperature features

## Variable importance

In [ ]:
# Get numerical feature importances
importances = list(rf.feature_importances_)
# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]
# Sorting the second element in each tuple
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances]

In [ ]:
# list of x locations for plotting
x_values = list(range(len(importances))) 

# List of features sorted from most to least important. Take the 2nd value from the list of tuple above
sorted_importances = [importance[1] for importance in feature_importances]
# Take the 1st value from the list of tuple above
sorted_features = [importance[0] for importance in feature_importances]

# Cumulative importances
cumulative_importances = np.cumsum(sorted_importances)

plt.figure(figsize=(10,8))

# Make a line graph
plt.plot(x_values, cumulative_importances, 'b-')

# Draw line at 95% of importance retained
plt.hlines(y = 0.95, xmin=0, xmax=len(sorted_importances), color = 'r', linestyles = 'dotted')

# Format x ticks and labels
plt.xticks(x_values, sorted_features, rotation = 'vertical')

# Axis labels and title
plt.xlabel('Variable') 
plt.ylabel('Cumulative Importance')
plt.title('Cumulative Importances')

This is the contribution to the overall importance of each feature. The red dotted line is set at 95% of total importance accounted for.

In [ ]:
# Find number of features for cumulative importance of 95%
# Add 1 because Python is zero-indexed
print('Number of features for 95% importance:', np.where(cumulative_importances > 0.95)[0][0] + 1, "on", len(feature_list), "total.")

In [ ]:
print(sorted_features)

### Consideration

First 3:
- T2 : temperature in lviign room area
- Press_mm_hg : is humidty in living room area in %
- T3 : temperature laundry

Remark: This is the prove on my small_tree.png analysis -> "there is a prevalence of temperature features"

Last 3:
- Visibilitiy: not surprising
- Random variable 1: not surprising
- Random variable 2: not surprising

### Let's make a Forest Tree with the most 3 important variables

In [ ]:
# New random forest with only the two most important variables
#rf_most_important = RandomForestRegressor(n_estimators= 100, random_state=1234)
# Extract the two most important features
#important_indices = [feature_list.index('T2'), feature_list.index('Press_mm_hg'), feature_list.index('T3')]
#train_important = train_features[:, important_indices]
#test_important = test_features[:, important_indices]
# Train the random forest
#rf_most_important.fit(train_important, train_labels)
# Make predictions and determine the error
#predictions = rf_most_important.predict(test_important)
#errors = abs(predictions - test_labels)
# Display the performance metrics
#print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')
#mape = np.mean(100 * (errors / test_labels))
#accuracy = 100 - mape
#print('Accuracy:', round(accuracy, 2), '%.')


Super! This means if we want to reduce the power computational stress we can work with top 3 features without losing model accuracy. 

### Let's try with the 5 most important feature and let's see what changes...

In [ ]:
# New random forest with only the two most important variables
#rf_most_important = RandomForestRegressor(n_estimators= 100, random_state=1234)
# Extract the two most important features
#important_indices = [feature_list.index('T2'), feature_list.index('Press_mm_hg'), feature_list.index('T3'), feature_list.index('T3'), feature_list.index('T8'), feature_list.index('Td')]
#train_important = train_features[:, important_indices]
#test_important = test_features[:, important_indices]
# Train the random forest
#rf_most_important.fit(train_important, train_labels)
# Make predictions and determine the error
#predictions = rf_most_important.predict(test_important)
#errors = abs(predictions - test_labels)
# Display the performance metrics
#print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')
#mape = np.mean(100 * (errors / test_labels))
#accuracy = 100 - mape
#print('Accuracy:', round(accuracy, 2), '%.')



That's the the prove we must use the whole model's feature. Ah, by the way, this is trivial, since the difference in every weighted features is quite small.

# Improve the RF

In machine learning, a hyperparameter is a parameter whose value is set before the learning process begins. By contrast, the values of other parameters are derived via training. Hyperparameter tuning relies more on experimental results than theory, and thus the best method to determine the optimal settings is to try many different combinations evaluate the performance of each model. However, evaluating each model only on the training set can lead to one of the most fundamental problems in machine learning: overfitting.

### Restrict to the Most Important Features

These were the six features required to reach a total feature importance of 95% in the first improving random forest notebook. We will use only these features in order to speed up the model.

In [ ]:
# Names of five importances accounting for 95% of total importance
important_feature_names = ['T2', 'RH_1', 'RH_8', 'RH_9', 'RH_out', 'RH_3', 'RH_5', 'T8', 'Press_mm_hg', 
                           'T3', 'T4', 'RH_4', 'T6', 'RH_7', 'T1', 'RH_2', 'T5', 'RH_6', 'T7', 'T_out',
                           'Tdewpoint', 'T9']

# Find the columns of the most important features
important_indices = [feature_list.index(feature) for feature in important_feature_names]

# Create training and testing sets with only the important features
important_train_features = train_features[:, important_indices]
important_test_features = test_features[:, important_indices]

# Sanity check on operations
print('Important train features shape:', important_train_features.shape)
print('Important test features shape:', important_test_features.shape)

In [ ]:
# Use only the most important features
train_features = important_train_features[:]
test_features = important_test_features[:]

# Update feature list for visualizations
feature_list = important_feature_names[:]

### Examine the Default Random Forest to Determine Parameters¶

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(random_state = 42)

from pprint import pprint

# Look at parameters used by our current forest
print('Parameters currently in use:\n')
pprint(rf.get_params())

## Cross Validation - KFold CV

### Random Search Cross Validation

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)] #10
# Number of features to consider at every split
max_features = ['auto', 'sqrt'] #2
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 100, num = 10)] #10
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
pprint(random_grid)

There are 2 * 11 * 2 * 3 * 3 * 10 = 3960 settings.
- Will my 1,3 GHz Intel Core i5 survive?
- Will be worth it?

In [ ]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator=rf, param_distributions=random_grid,
                              n_iter = 100, scoring='neg_mean_absolute_error', 
                              cv = 3, verbose=2, random_state=42, n_jobs=-1)

# Fit the random search model
rf_random.fit(train_features, train_labels)

In [ ]:
rf_random.best_params_

## Evaluate function

In [ ]:
def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    errors = abs(predictions - test_labels)
    mape = 100 * np.mean(errors / test_labels)
    accuracy = 100 - mape
    print('Model Performance')
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))

### Evaluate the default model

In [ ]:
base_model = RandomForestRegressor(n_estimators = 1000, random_state = 42)
base_model.fit(train_features, train_labels)
evaluate(base_model, test_features, test_labels)

### Evaluate the best random search model

In [ ]:
best_random = rf_random.best_estimator_
evaluate(best_random, test_features, test_labels)

## Grid Search

In [ ]:
from sklearn.model_selection import GridSearchCV

# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [True],
    'max_depth': [80, 90, 100, 110],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 1000]
}

# Create a based model
rf = RandomForestRegressor()

# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                           scoring = 'neg_mean_absolute_error', cv = 3, 
                           n_jobs = -1, verbose = 2)


In [ ]:
# Fit the grid search to the data
grid_search.fit(train_features, train_labels)

In [ ]:
grid_search.best_params_

In [ ]:
best_grid = grid_search.best_estimator_
evaluate(best_grid, test_features, test_labels)

### Another Round of Grid Search¶

In [ ]:
param_grid = {
    'bootstrap': [True],
    'max_depth': [110, 120, None],
    'max_features': [3, 4],
    'min_samples_leaf': [5, 6, 7],
    'min_samples_split': [10],
    'n_estimators': [75, 100, 125]
}

# Create a based model
rf = RandomForestRegressor()

# Instantiate the grid search model
grid_search_ad = GridSearchCV(estimator = rf, param_grid = param_grid, 
                           scoring = 'neg_mean_absolute_error', cv = 3, 
                           n_jobs = -1, verbose = 2)

grid_search_ad.fit(train_features, train_labels)

In [ ]:
grid_search_ad.best_params_

In [ ]:
best_grid_ad = grid_search_ad.best_estimator_
evaluate(best_grid_ad, test_features, test_labels)

## Final model

In [ ]:
print('Model Parameters:\n')
pprint(best_grid.get_params())
print('\n')
evaluate(best_grid, test_features, test_labels)

In [ ]:
print("hello world")

In [ ]:
# Adding column to mark weekdays (0) and weekends(1) for time series evaluation , 
# 
# The day of the week with Monday=0, Sunday=6.
# Return the day of the week. It is assumed the week starts on Monday, which is denoted by 0 and ends on Sunday which is denoted by 6. 
# This method is available on both Series with datetime values (using the dt accessor) or DatetimeIndex.

df['WEEKDAY'] = ((pd.to_datetime(df['date']).dt.dayofweek)//5 == 1).astype(float)
# There are 5472 weekend recordings 
df['WEEKDAY'].value_counts()

In [ ]:
# Find rows with weekday 

temp_weekend =  data[data['WEEKDAY'] == 1]

# To understand the timeseries variation of the applaince energy consumption
visData = go.Scatter( x= temp_weekend.date  ,  mode = "lines", y = temp_weekend.Appliances )
layout = go.Layout(title = 'Appliance energy consumption measurement on weekend' , xaxis=dict(title='Date'), yaxis=dict(title='(Wh)'))
fig = go.Figure(data=[visData],layout=layout)

iplot(fig)

In [ ]:
# Histogram of all the features to understand the distribution
feature_vars.hist(bins = 20 , figsize= (12,16)) ;

In [ ]:
# focussed displots for RH_6 , RH_out , Visibility , Windspeed due to irregular distribution
f, ax = plt.subplots(2,2,figsize=(12,8))
vis1 = sns.distplot(feature_vars["RH_6"],bins=10, ax= ax[0][0])
vis2 = sns.distplot(feature_vars["RH_out"],bins=10, ax=ax[0][1])
vis3 = sns.distplot(feature_vars["Visibility"],bins=10, ax=ax[1][0])
vis4 = sns.distplot(feature_vars["Windspeed"],bins=10, ax=ax[1][1])

In [ ]:
# Distribution of values in Applainces column
f = plt.figure(figsize=(12,5))
plt.xlabel('Appliance consumption in Wh')
plt.ylabel('Frequency')
sns.distplot(target_vars , bins=10 ) ;

### Observations 

1. Temperature - All the columns follow normal distribution except T9
2. Humidity - All columns follow normal distribution except RH_6 and RH_out , primarly because these sensors are outside the house 
3. Appliance - This column is postively skewed , most the values are around mean 100 Wh . There are outliers in this column 
4. Visibilty - This column is negatively skewed
5. Windspeed - This column is postively skewed


In [ ]:
#Appliance column range with consumption less than 200 Wh
print('Percentage of the appliance consumption is less than 200 Wh')
print(((target_vars[target_vars <= 200].count()) / (len(target_vars)))*100 )

### Correlation Plots

In [ ]:
# Use the weather , temperature , applainces and random column to see the correlation
train_corr = train[col_temp + col_hum + col_weather +col_target+col_randoms]
corr = train_corr.corr()
# Mask the repeated values
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True
  
f, ax = plt.subplots(figsize=(16, 14))
#Generate Heat Map, allow annotations and place floats in map
sns.heatmap(corr, annot=True, fmt=".2f" , mask=mask,)
    #Apply xticks
plt.xticks(range(len(corr.columns)), corr.columns);
    #Apply yticks
plt.yticks(range(len(corr.columns)), corr.columns)
    #show plot
plt.show()

In [ ]:
def get_redundant_pairs(df):
    '''Get diagonal and lower triangular pairs of correlation matrix'''
    pairs_to_drop = set()
    cols = df.columns
    for i in range(0, df.shape[1]):
        for j in range(0, i+1):
            pairs_to_drop.add((cols[i], cols[j]))
    return pairs_to_drop

# Function to get top correlations 

def get_top_abs_correlations(df, n=5):
    au_corr = df.corr().abs().unstack()
    labels_to_drop = get_redundant_pairs(df)
    au_corr = au_corr.drop(labels=labels_to_drop).sort_values(ascending=False)
    return au_corr[0:n]

print("Top Absolute Correlations")
print(get_top_abs_correlations(train_corr, 40))

### Observations based on correlation plot

1. Temperature - All the temperature variables from T1-T9 and T_out have positive correlation with the target Appliances . For the indoortemperatures, the correlations are high as expected, since the ventilation is driven by the HRV unit and minimizes air temperature differences between rooms. Four columns have a high degree of correlation with T9 - T3,T5,T7,T8 also T6 & T_Out has high correlation (both temperatures from outside) . Hence T6 & T9 can be removed from training set as information provided by them can be provided by other fields.

2. Weather attributes - Visibility, Tdewpoint, Press_mm_hg  have low correlation values

3. Humidity - There are no significantly high  correlation cases (> 0.9) for humidity sensors.

4. Random variables have no role to play


# Data Pre Processing

In [ ]:
#Split training dataset into independent and dependent varibales
train_X = train[feature_vars.columns] # features
train_y = train[target_vars.columns]# target

In [ ]:
#Split testing dataset into independent and dependent varibales
test_X = test[feature_vars.columns] # features
test_y = test[target_vars.columns] # target

In [ ]:
# Due to conlusion made above below columns are removed
train_X.drop(["rv1","rv2","Visibility","T6","T9"],axis=1 , inplace=True)

In [ ]:
# Due to conlusion made above below columns are removed
test_X.drop(["rv1","rv2","Visibility","T6","T9"], axis=1, inplace=True)

In [ ]:
train_X.columns

In [ ]:
test_X.columns

In [ ]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()

# Create test and training set by including Appliances column

train = train[list(train_X.columns.values) + col_target ] # add target to fit it as well features variable 

test = test[list(test_X.columns.values) + col_target ] # add target to fit it as well features variable 

# Create dummy test and training set to hold scaled values
# sc = standardize features by removing the mean and scaling to unit variance
# because estimators might behave badly if the individual features do not more or less look 
# like standard normally distributed data

sc_train = pd.DataFrame(columns=train.columns , index=train.index)

sc_train[sc_train.columns] = sc.fit_transform(train) # fit to data, then transform it.

sc_test= pd.DataFrame(columns=test.columns , index=test.index)

sc_test[sc_test.columns] = sc.fit_transform(test) # fit to data, then transform it.


In [ ]:
sc_train.head()

In [ ]:
sc_test.head()

In [ ]:
# One fitted, we can remove Appliances (target) column from traininig set

train_X =  sc_train.drop(['Appliances'] , axis=1)
train_y = sc_train['Appliances']

test_X =  sc_test.drop(['Appliances'] , axis=1)
test_y = sc_test['Appliances']

In [ ]:
train_X.head()

In [ ]:
train_y.head()

# Model Implementation

We will be looking at following Algorithms 

**Improved and not improved Linear regression models**

1.Simple Linear Regression

2.Ridge regression 

3.Lasso regression 

**Support Vector Machine**

3.Support vector regression 

**Nearest neighbour Regressor**

4.KNeighborsRegressor

**Ensmble models**

5.Random Forest Regressor

6.Gradient Boosting Regressor

7.ExtraTrees Regressor



In [ ]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.preprocessing import PolynomialFeatures 
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
import xgboost as xgb
from sklearn import neighbors
from sklearn.svm import SVR


In [ ]:
models = [
           ['Linear: ', LinearRegression()],
           ['Lasso: ', Lasso()],
           ['Ridge: ', Ridge()],
           ['KNeighborsRegressor: ',  neighbors.KNeighborsRegressor()],
           ['SVR:' , SVR(kernel='rbf')],
           ['RandomForest ',RandomForestRegressor()],
           ['ExtraTreeRegressor :',ExtraTreesRegressor()],
           ['GradientBoostingClassifier: ', GradientBoostingRegressor()] ,
           ['XGBRegressor: ', xgb.XGBRegressor()] ,
         ]


In [ ]:
# Run all the proposed models and update the information in a list model_data
import time
from math import sqrt
from sklearn.metrics import mean_squared_error

model_data = []
for name,curr_model in models :
    curr_model_data = {}
    curr_model.random_state = 78
    curr_model_data["Name"] = name
    start = time.time()
    curr_model.fit(train_X,train_y)
    end = time.time()
    curr_model_data["Train_Time"] = end - start
    curr_model_data["Train_R2_Score"] = metrics.r2_score(train_y,curr_model.predict(train_X))
    curr_model_data["Test_R2_Score"] = metrics.r2_score(test_y,curr_model.predict(test_X))
    curr_model_data["Test_RMSE_Score"] = sqrt(mean_squared_error(test_y,curr_model.predict(test_X)))
    model_data.append(curr_model_data)

In [ ]:
model_data

In [ ]:
# Convert list to dataframe
df_models = pd.DataFrame(model_data)

In [ ]:
df_models

In [ ]:
df.plot(x="Name", y=['Test_R2_Score' , 'Train_R2_Score' , 'Test_RMSE_Score'], kind="bar" , title = 'R2 Score Results' , figsize= (10,8)) ;

### Obervations

1. Best results over test set are given by Extra Tree Regressor with R2 score of 0.57
2. Least RMSE score is also by Extra Tree Regressor 0.65
2. Lasso regularization over Linear regression was worst performing model


# Parameter Tuning

In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid = [{
              'max_depth': [80, 150, 200,250],
              'n_estimators' : [100,150,200,250],
              'max_features': ["auto", "sqrt", "log2"]
            }]
reg = ExtraTreesRegressor(random_state=40)
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = reg, param_grid = param_grid, cv = 5, n_jobs = -1 , scoring='r2' , verbose=2)
grid_search.fit(train_X, train_y)

In [ ]:
# Tuned parameter set
grid_search.best_params_

In [ ]:
# Best possible parameters for ExtraTreesRegressor
grid_search.best_estimator_

In [ ]:
# R2 score on training set with tuned parameters

grid_search.best_estimator_.score(train_X,train_y)

In [ ]:
# R2 score on test set with tuned parameters
grid_search.best_estimator_.score(test_X,test_y)

In [ ]:
# RMSE score on test set with tuned parameters

np.sqrt(mean_squared_error(test_y, grid_search.best_estimator_.predict(test_X)))

### Observations

Based on parameter tunning step we can see that 

1. Best possible parameter combination are - 'max_depth': 80, 'max_features': 'sqrt', 'n_estimators': 200

    
2. Training set  R2 score of 1.0 may be signal of overfitting on training set 


3. Test set R2 score is 0.63 improvement over 0.57 achieved using untuned model


4. Test set RMSE score is 0.60 improvement over 0.65 achieved using untuned model 




### Feature Importance 

In [ ]:
# Get sorted list of features in order of importance
feature_indices = np.argsort(grid_search.best_estimator_.feature_importances_)

In [ ]:
importances = grid_search.best_estimator_.feature_importances_
indices = np.argsort(importances)[::-1]
names = [train_X.columns[i] for i in indices]
# Create plot
plt.figure(figsize=(10,6))

# Create plot title
plt.title("Feature Importance")

# Add bars
plt.bar(range(train_X.shape[1]), importances[indices])

# Add feature names as x-axis labels
plt.xticks(range(train_X.shape[1]), names, rotation=90)

# Show plot
plt.show()

In [ ]:
# Get top 5 most important feature 
names[0:5]

In [ ]:
# Get 5 least important feature 
names[-5:]

In [ ]:
# Reduce test & training set to 5 feature set
train_important_feature = train_X[names[0:5]]
test_important_feature = test_X[names[0:5]]

In [ ]:
# Clone the Gridsearch model with his parameter and fit on reduced dataset

from sklearn.base import clone
cloned_model = clone(grid_search.best_estimator_)
cloned_model.fit(train_important_feature , train_y)

In [ ]:
# Reduced dataset scores 

print('Training set R2 Score - ', metrics.r2_score(train_y,cloned_model.predict(train_important_feature)))
print('Testing set R2 Score - ', metrics.r2_score(test_y,cloned_model.predict(test_important_feature)))
print('Testing set RMSE Score - ', np.sqrt(mean_squared_error(test_y, cloned_model.predict(test_important_feature))))


### Observations 

1. Based on parameter tunning step we can see that 

    a. 5 most important features are - 'RH_out', 'RH_8', 'RH_1', 'T3', 'RH_3'
    
    b. 5 least important features are - 'T7','Tdewpoint','Windspeed','T1','T5'
    

2. As can be observed with R2 Score , compared to Tuned model 0.63 the R2 score has come down to 0.47 which is decrease of 16% .


3. The reduction in R2 score is high and we should not use reduced feature set for this data set

# Conclusion

1. The best Algorithm to use for this dataset Extra Trees Regressor

2. The untuned model was able to explain 57% of variance on test set .

3. The tuned model was able to explain 63% of varaince on tese set which is improvement of 10%

4. The final model had 22 features 

5. Feature reduction was not able to add to better R2 score 

